# Pretraining on WT103

In [ ]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

In [ ]:
#export
from exp.nb_12a import *

## Data

One time donwload

In [ ]:
#path = datasets.Config().data_path()
#version = '2' #'103'

In [ ]:
#! wget https://s3.amazonaws.com/research.metamind.io/wikitext/wikitext-{version}-v1.zip -P {path}
#! unzip -q -n {path}/wikitext-{version}-v1.zip  -d {path}
#! mv {path}/wikitext-{version}/wiki.train.tokens {path}/wikitext-{version}/train.txt
#! mv {path}/wikitext-{version}/wiki.valid.tokens {path}/wikitext-{version}/valid.txt
#! mv {path}/wikitext-{version}/wiki.test.tokens {path}/wikitext-{version}/test.txt

Split the articles

In [ ]:
path = datasets.Config().data_path()/'wikitext-2'

In [ ]:
def istitle(line):
    return len(re.findall(r'^ = [^=]* = $', line)) != 0

In [ ]:
def read_wiki(filename):
    articles = []
    with open(filename, encoding='utf8') as f:
        lines = f.readlines()
    current_article = ''
    for i,line in enumerate(lines):
        current_article += line
        if i < len(lines)-2 and lines[i+1] == ' \n' and istitle(lines[i+2]):
            current_article = current_article.replace('<unk>', UNK)
            articles.append(current_article)
            current_article = ''
    current_article = current_article.replace('<unk>', UNK)
    articles.append(current_article)
    return articles

In [ ]:
train = TextList(read_wiki(path/'train.txt'), path=path) #+read_file(path/'test.txt')
valid = TextList(read_wiki(path/'valid.txt'), path=path)

In [ ]:
len(train), len(valid)

(601, 60)

In [ ]:
sd = SplitData(train, valid)

In [ ]:
ll = label_by_func(sd, lambda x: 0, proc_x = [TokenizeProcessor(), NumericalizeProcessor()])

In [ ]:
pickle.dump(ll, open(path/'ld.pkl', 'wb'))

In [ ]:
bs,bptt = 64,70
data = lm_databunchify(ll, bs, bptt)

In [ ]:
vocab = ll.train.x.processors[1].vocab

## Model

In [ ]:
dps = np.array([0.25, 0.1, 0.2, 0.02, 0.15]) * 0.25

In [ ]:
emb_sz, nh, nl = 300, 300, 2
model = get_language_model(len(vocab), emb_sz, nh, nl, 0, input_p=dps[0], output_p=dps[1], weight_p=dps[2], 
                           embed_p=dps[3], hidden_p=dps[4])

In [ ]:
lr = 5e-3
sched_lr  = combine_scheds([0.5,0.5], cos_1cycle_anneal(lr/10.,lr, 0))
sched_mom = combine_scheds([0.5,0.5], cos_1cycle_anneal(0.8,0.7, 0.8))

In [ ]:
cbs = [partial(AvgStatsCallback,accuracy_flat),
       CudaCallback,
       Recorder,
       partial(GradientClipping, clip=0.1),
       partial(ParamScheduler, 'lr', sched_lr),
       partial(ParamScheduler, 'mom', sched_mom),
       partial(RNNTrainer, alpha=2., beta=1.),
       ProgressCallback]

In [ ]:
learn = Learner(model, data, cross_entropy_flat, lr=5e-3, cb_funcs=cbs, opt_func=adam_opt)

In [ ]:
learn.fit(10)

In [ ]:
torch.save(learn.model, path/'pretrained.pth')
pickle.dump(vocab, open(path/'vocab.pkl', 'rb'))